In [10]:
import requests
from lxml import html
from pymongo import MongoClient

In [11]:
url = 'https://www.imdb.com/chart/moviemeter/?ref_=nv_mv_mpm'
header = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36'
}
resp = requests.get(url=url, headers=header)
resp.status_code

200

In [12]:
# парсинг дерева lxml из содержимого ответа
tree = html.fromstring(html=resp.content)

In [13]:
tree.xpath('//div[@data-testid="chart-layout-main-column"]/ul/li')


[<Element li at 0x19595a32b10>,
 <Element li at 0x195959f49f0>,
 <Element li at 0x195959f4900>,
 <Element li at 0x195959f5e40>,
 <Element li at 0x195959f4860>,
 <Element li at 0x195959f4a90>,
 <Element li at 0x195959f5a30>,
 <Element li at 0x195959f5a80>,
 <Element li at 0x195959f5b20>,
 <Element li at 0x195959f5b70>,
 <Element li at 0x195959f5ad0>,
 <Element li at 0x195959f5c10>,
 <Element li at 0x195959f5c60>,
 <Element li at 0x195959f5bc0>,
 <Element li at 0x195959f5d00>,
 <Element li at 0x195959f5d50>,
 <Element li at 0x195959f5cb0>,
 <Element li at 0x195959f5da0>,
 <Element li at 0x195959f5620>,
 <Element li at 0x195959f56c0>,
 <Element li at 0x195959f5800>,
 <Element li at 0x195959f57b0>,
 <Element li at 0x195959f5760>,
 <Element li at 0x195959f5710>,
 <Element li at 0x195959f5670>,
 <Element li at 0x195959f59e0>,
 <Element li at 0x195959f5990>,
 <Element li at 0x195959f5940>,
 <Element li at 0x195959f58f0>,
 <Element li at 0x195959f58a0>,
 <Element li at 0x195959f6930>,
 <Elemen

In [14]:
# найдем на странице список фильмов
movies = tree.xpath('//div[@data-testid="chart-layout-main-column"]/ul/li')

all_movies = []

# в каждом фильме из списка будем искать нужные нам данные
for movie in movies:
    pos = movie.xpath(".//descendant::div[@aria-label]/span/text()")
    pos_charge = pos[0] if len(pos)>0 else '-'

    # сделаем словарь для каждого фильма с ключами:
    # название, год выпуска, рейтинг, рост/падение рейтинга, изменение рейтинга
    m = {
        'name' : movie.xpath(".//descendant::h3/text()")[0],
        'release_year' : int(movie.xpath('.//descendant::div[@class="sc-b189961a-7 feoqjK cli-title-metadata"]/span/text()')[0]),
        'position' : int(movie.xpath(".//descendant::div[@aria-label]/text()")[0]),
        'titlemeter' : ' '.join(movie.xpath(".//descendant::div[@aria-label]/span/@aria-label")[0].split(' ')[:2]),
        'position_charge' : pos_charge}

    all_movies.append(m)

In [15]:
all_movies

[{'name': 'Падение империи',
  'release_year': 2024,
  'position': 1,
  'titlemeter': 'No rank',
  'position_charge': '-'},
 {'name': 'Претенденты',
  'release_year': 2024,
  'position': 2,
  'titlemeter': 'Moved up',
  'position_charge': '9'},
 {'name': 'Мятежная Луна, часть 2: Оставляющая шрамы шрамы',
  'release_year': 2024,
  'position': 3,
  'titlemeter': 'Moved up',
  'position_charge': '7'},
 {'name': 'Эбигейл',
  'release_year': 2024,
  'position': 4,
  'titlemeter': 'Moved up',
  'position_charge': '1'},
 {'name': 'Министерство неджентльменских дел',
  'release_year': 2024,
  'position': 5,
  'titlemeter': 'Moved down',
  'position_charge': '2'},
 {'name': 'Дэдпул и Россомаха',
  'release_year': 2024,
  'position': 6,
  'titlemeter': 'Moved up',
  'position_charge': '50'},
 {'name': 'Дюна: Часть вторая',
  'release_year': 2024,
  'position': 7,
  'titlemeter': 'Moved down',
  'position_charge': '5'},
 {'name': 'Полночь с дьяволом',
  'release_year': 2023,
  'position': 8,
  't

In [16]:
len(all_movies)

100

Добавим полученный список в базу Mongo

In [17]:
# напишем функцию добавления списка фильмов в базу
def insert_to_db(list_movies):
    client = MongoClient("mongodb://localhost:27017")
    db = client["imdb_movies"]
    collection = db["top_movies"]
    collection.insert_many(list_movies)
    client.close()

In [18]:
insert_to_db(all_movies)